In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import xgboost as xgb 

In [2]:
df = pd.read_csv("Liver Dataset.csv")

In [3]:
df.head()

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,CL,D-penicillamine,19581,F,N,N,Y,N,0.3,279.0,2.96,84.0,1500.8,99.43,109.0,293.0,10.2,4
1,7326,41,C,D-penicillamine,22880,F,NaN,N,NaN,N,0.3,NaN,2.96,NaN,1835.4,26.35,131.0,308.0,10.8,1
2,7254,297,D,NaN,27957,F,N,N,NaN,N,0.3,328.0,2.64,4.0,NaN,NaN,116.0,194.0,10.3,3
3,3135,1872,C,D-penicillamine,21111,F,NaN,Y,Y,N,0.3,302.0,2.02,49.0,NaN,26.35,NaN,NaN,10.5,4
4,2483,939,CL,D-penicillamine,18061,F,NaN,NaN,NaN,N,0.5,344.0,3.11,91.0,NaN,104.56,NaN,306.0,11.4,2


In [4]:
df = df.drop_duplicates()

df["SGOT"].fillna("68.569121",  inplace = True)
df["Tryglicerides"].fillna("68.569121",  inplace = True)
df["Platelets"].fillna("68.569121",  inplace = True)
df["Prothrombin"].fillna("68.569121",  inplace = True)
df=df["Drug"].fillna("NaN",  inplace = True)
df =df["Ascites"].fillna("NaN",  inplace = True)
df = df["Hepatomegaly"].fillna("NaN",  inplace = True)
df = df["Spiders"].fillna("NaN",  inplace = True)

In [5]:
df.fillna(df.mean(), inplace=True)

C:\Users\pahar\AppData\Local\Temp\ipykernel_16748\820435583.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [6]:
df.isnull().sum()

ID                  0
N_Days              0
Status              0
Drug             2025
Age                 0
Sex                 0
Ascites          2246
Hepatomegaly     2427
Spiders          2590
Edema               0
Bilirubin           0
Cholesterol         0
Albumin             0
Copper              0
Alk_Phos            0
SGOT                0
Tryglicerides       0
Platelets           0
Prothrombin         0
Stage               0
dtype: int64

In [7]:
df["Drug"].unique()
df["Ascites"].unique()
df["Sex"].unique()
df["Hepatomegaly"].unique()
df["Spiders"].unique()

array(['Y', nan, 'N'], dtype=object)

In [8]:
df["Age"].describe()

count     6800.000000
mean     22863.223529
std       4671.135682
min       9598.000000
25%      19416.500000
50%      23367.000000
75%      27125.500000
max      28650.000000
Name: Age, dtype: float64

In [9]:
df["Stage"].unique()

array([4, 1, 3, 2], dtype=int64)

In [10]:
# Replace values in the 'Drug' column
df['Drug'].replace({'D-penicillamine': 0, 'Placebo': 1}, inplace=True)
df

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,CL,0.0,19581,F,N,N,Y,N,0.3,279.000000,2.96,84.000000,1500.800000,99.430000,109.000000,293.000000,10.2,4
1,7326,41,C,0.0,22880,F,NaN,N,NaN,N,0.3,294.575831,2.96,68.569121,1835.400000,26.350000,131.000000,308.000000,10.8,1
2,7254,297,D,NaN,27957,F,N,N,NaN,N,0.3,328.000000,2.64,4.000000,1802.951464,79.554427,116.000000,194.000000,10.3,3
3,3135,1872,C,0.0,21111,F,NaN,Y,Y,N,0.3,302.000000,2.02,49.000000,1802.951464,26.350000,133.857823,293.306097,10.5,4
4,2483,939,CL,0.0,18061,F,NaN,NaN,NaN,N,0.5,344.000000,3.11,91.000000,1802.951464,104.560000,133.857823,306.000000,11.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,4622,1762,C,1.0,20909,M,NaN,N,NaN,N,0.9,360.000000,2.43,68.569121,3444.400000,131.590000,130.000000,314.000000,12.9,1
6796,4446,2955,C,1.0,28650,F,Y,N,N,N,0.3,294.575831,2.99,43.000000,1802.951464,79.554427,238.000000,399.000000,10.6,4
6797,4082,584,C,NaN,24102,F,N,Y,NaN,N,0.3,218.000000,2.79,68.569121,1802.951464,26.350000,123.000000,103.000000,9.3,1
6798,2248,1426,CL,0.0,19791,F,N,NaN,Y,N,0.3,294.575831,1.96,12.000000,289.000000,72.950000,134.000000,354.000000,10.6,2


In [11]:
df['Status'].replace({'CL': 0, 'C': 1, 'D' : 2}, inplace=True)
df

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,0,0.0,19581,F,N,N,Y,N,0.3,279.000000,2.96,84.000000,1500.800000,99.430000,109.000000,293.000000,10.2,4
1,7326,41,1,0.0,22880,F,NaN,N,NaN,N,0.3,294.575831,2.96,68.569121,1835.400000,26.350000,131.000000,308.000000,10.8,1
2,7254,297,2,NaN,27957,F,N,N,NaN,N,0.3,328.000000,2.64,4.000000,1802.951464,79.554427,116.000000,194.000000,10.3,3
3,3135,1872,1,0.0,21111,F,NaN,Y,Y,N,0.3,302.000000,2.02,49.000000,1802.951464,26.350000,133.857823,293.306097,10.5,4
4,2483,939,0,0.0,18061,F,NaN,NaN,NaN,N,0.5,344.000000,3.11,91.000000,1802.951464,104.560000,133.857823,306.000000,11.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,4622,1762,1,1.0,20909,M,NaN,N,NaN,N,0.9,360.000000,2.43,68.569121,3444.400000,131.590000,130.000000,314.000000,12.9,1
6796,4446,2955,1,1.0,28650,F,Y,N,N,N,0.3,294.575831,2.99,43.000000,1802.951464,79.554427,238.000000,399.000000,10.6,4
6797,4082,584,1,NaN,24102,F,N,Y,NaN,N,0.3,218.000000,2.79,68.569121,1802.951464,26.350000,123.000000,103.000000,9.3,1
6798,2248,1426,0,0.0,19791,F,N,NaN,Y,N,0.3,294.575831,1.96,12.000000,289.000000,72.950000,134.000000,354.000000,10.6,2


In [12]:
df['Sex'].replace({'F': 0, 'M': 1}, inplace=True)
df

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,0,0.0,19581,0,N,N,Y,N,0.3,279.000000,2.96,84.000000,1500.800000,99.430000,109.000000,293.000000,10.2,4
1,7326,41,1,0.0,22880,0,NaN,N,NaN,N,0.3,294.575831,2.96,68.569121,1835.400000,26.350000,131.000000,308.000000,10.8,1
2,7254,297,2,NaN,27957,0,N,N,NaN,N,0.3,328.000000,2.64,4.000000,1802.951464,79.554427,116.000000,194.000000,10.3,3
3,3135,1872,1,0.0,21111,0,NaN,Y,Y,N,0.3,302.000000,2.02,49.000000,1802.951464,26.350000,133.857823,293.306097,10.5,4
4,2483,939,0,0.0,18061,0,NaN,NaN,NaN,N,0.5,344.000000,3.11,91.000000,1802.951464,104.560000,133.857823,306.000000,11.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,4622,1762,1,1.0,20909,1,NaN,N,NaN,N,0.9,360.000000,2.43,68.569121,3444.400000,131.590000,130.000000,314.000000,12.9,1
6796,4446,2955,1,1.0,28650,0,Y,N,N,N,0.3,294.575831,2.99,43.000000,1802.951464,79.554427,238.000000,399.000000,10.6,4
6797,4082,584,1,NaN,24102,0,N,Y,NaN,N,0.3,218.000000,2.79,68.569121,1802.951464,26.350000,123.000000,103.000000,9.3,1
6798,2248,1426,0,0.0,19791,0,N,NaN,Y,N,0.3,294.575831,1.96,12.000000,289.000000,72.950000,134.000000,354.000000,10.6,2


In [13]:
df['Edema'].replace({'N': 0, 'Y': 1, 'S' : 2}, inplace=True)
df['Ascites'].replace({'N': 0, 'Y': 1}, inplace=True)
df['Hepatomegaly'].replace({'N': 0, 'Y': 1}, inplace=True)
df['Spiders'].replace({'N': 0, 'Y': 1}, inplace=True)
df

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7135,1654,0,0.0,19581,0,0.0,0.0,1.0,0,0.3,279.000000,2.96,84.000000,1500.800000,99.430000,109.000000,293.000000,10.2,4
1,7326,41,1,0.0,22880,0,NaN,0.0,NaN,0,0.3,294.575831,2.96,68.569121,1835.400000,26.350000,131.000000,308.000000,10.8,1
2,7254,297,2,NaN,27957,0,0.0,0.0,NaN,0,0.3,328.000000,2.64,4.000000,1802.951464,79.554427,116.000000,194.000000,10.3,3
3,3135,1872,1,0.0,21111,0,NaN,1.0,1.0,0,0.3,302.000000,2.02,49.000000,1802.951464,26.350000,133.857823,293.306097,10.5,4
4,2483,939,0,0.0,18061,0,NaN,NaN,NaN,0,0.5,344.000000,3.11,91.000000,1802.951464,104.560000,133.857823,306.000000,11.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,4622,1762,1,1.0,20909,1,NaN,0.0,NaN,0,0.9,360.000000,2.43,68.569121,3444.400000,131.590000,130.000000,314.000000,12.9,1
6796,4446,2955,1,1.0,28650,0,1.0,0.0,0.0,0,0.3,294.575831,2.99,43.000000,1802.951464,79.554427,238.000000,399.000000,10.6,4
6797,4082,584,1,NaN,24102,0,0.0,1.0,NaN,0,0.3,218.000000,2.79,68.569121,1802.951464,26.350000,123.000000,103.000000,9.3,1
6798,2248,1426,0,0.0,19791,0,0.0,NaN,1.0,0,0.3,294.575831,1.96,12.000000,289.000000,72.950000,134.000000,354.000000,10.6,2


In [14]:
# Define features and target variable
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an imputer with the strategy to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create and train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_imputed, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_imputed)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"Model Accuracy: {accuracy_percentage:.2f}%")

Model Accuracy: 49.78%


In [15]:
# Replace values in the 'Stage' column
df['Stage'].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)

In [16]:
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train an XGBoost classifier
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=4, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"XGBoost Model Accuracy: {accuracy_percentage:.2f}%")

XGBoost Model Accuracy: 46.54%


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Load your preprocessed dataset
# Assuming you have already preprocessed your dataset and assigned it to df

# Define features and target variable
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an imputer with the strategy to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to your feature matrices
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create and train a Gradient Boosting Classifier
clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train_imputed, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_imputed)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"Gradient Boosting Model Accuracy: {accuracy_percentage:.2f}%")

Gradient Boosting Model Accuracy: 50.07%


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Load your preprocessed dataset
# Assuming you have already preprocessed your dataset and assigned it to df

# Define features and target variable
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an imputer with the strategy to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to your feature matrices
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create and train a Support Vector Machine (SVM) classifier
clf = SVC(random_state=42)
clf.fit(X_train_imputed, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_imputed)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"SVM Model Accuracy: {accuracy_percentage:.2f}%")

SVM Model Accuracy: 50.66%


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score

# Load your preprocessed dataset
# Assuming you have already preprocessed your dataset and assigned it to df

# Define features and target variable
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a simple feedforward neural network
model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model on the test set
_, accuracy = model.evaluate(X_test_scaled, y_test)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"Neural Network Model Accuracy: {accuracy_percentage:.2f}%")


Epoch 1/100
170/170 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.0702
Epoch 2/100
170/170 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.0699
Epoch 3/100
170/170 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0699
Epoch 4/100
170/170 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.0699
Epoch 5/100
170/170 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0699
Epoch 6/100
170/170 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.0699
Epoch 7/100
170/170 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0699
Epoch 8/100
170/170 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0699
Epoch 9/100
170/170 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.0699
Epoch 10/100
170/170 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0699
Epoch 11/100
170/17

In [20]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

# Load your preprocessed dataset
# Assuming you have already preprocessed your dataset and assigned it to df

# Define features and target variable
features = ['N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']
target = 'Stage'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an imputer with the strategy to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to your feature matrices
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train_imputed, label=y_train)

# Set hyperparameters for the LightGBM model
params = {
    'objective': 'multiclass',
    'num_class': 4,  # Number of classes
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the LightGBM model
clf = lgb.train(params, train_data, 100)

# Make predictions on the test set
y_pred = clf.predict(X_test_imputed, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_pred]

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to a percentage
accuracy_percentage = accuracy * 100

# Print the accuracy as a percentage
print(f"LightGBM Model Accuracy: {accuracy_percentage:.2f}%")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001017 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2472
[LightGBM] [Info] Number of data points in the train set: 5440, number of used features: 18
[LightGBM] [Info] Start training from score -2.661363
[LightGBM] [Info] Start training from score -1.527418
[LightGBM] [Info] Start training from score -1.633625
[LightGBM] [Info] Start training from score -0.658107
LightGBM Model Accuracy: 50.00%
